In [14]:
import numpy as np
import pandas as pd
import instrumentum

import lightgbm as ltb
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

from instrumentum.model_tuning.wrapper_optuna import OptunaSearchCV
from instrumentum.model_tuning._optuna_dispatchers import optuna_param_disp

RANDOM_SEED = 42

In [3]:
# Specify data paths
dataset = 'out/gestures.csv'

# Load Dataset

In [4]:
data = pd.read_csv(dataset, header=None) 

y = data.iloc[:, 0]
X = data.iloc[: , 1:]

y.value_counts()

random         1874
hand_closed     673
two_up          648
hang_in         623
hand_open       581
victory         530
call            525
one_up          416
machedici       414
ok              409
Name: 0, dtype: int64

In [5]:
classes = y.unique()
classes = {classes[i]: i for i in range(len(classes))}

y = y.map(classes)

X = X.to_numpy().astype(np.float32)
y = y.to_numpy().astype(np.float32)

classes

{'ok': 0,
 'victory': 1,
 'call': 2,
 'hang_in': 3,
 'one_up': 4,
 'two_up': 5,
 'hand_closed': 6,
 'hand_open': 7,
 'machedici': 8,
 'random': 9}

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=RANDOM_SEED)
# # y_train = y_train.reshape(len(y_train),1)
# # y_test = y_test.reshape(len(y_test),1)
# classes = len(np.unique(y))

In [ ]:
from sklearn.metrics import roc_auc_score


In [17]:
estimator = ltb.LGBMClassifier()

search_function = optuna_param_disp[estimator.__class__.__name__]
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)

os = OptunaSearchCV(
    estimator=estimator,
    scoring="roc_auc_ovo",
    cv=cv,
    search_space=search_function,
    n_iter=5,
)
os.fit(X_train, y_train)

[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
[LightGBM] [Fatal] Multiclass objective and metrics don't match
/opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------

ValueError: No trials are completed yet.

In [ ]:
import optuna

def _lgbmclassifier_default(trial: optuna.trial.Trial):
    params = {
        # "verbosity": -1,
        "boosting_type": trial.suggest_categorical(
            "boosting_type", ["gbdt", "dart", "goss"]
        ),
        "verbose": -1,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_leaves": trial.suggest_int("num_leaves", 5, 500, step=5),
        "max_depth": trial.suggest_int("max_depth", 1, 15),
        "learning_rate": trial.suggest_float(
            "learning_rate", 1e-5, 1.0, log=True
        ),
        "colsample_bytree": trial.suggest_uniform(
            "colsample_bytree", 0.0, 1.0
        ),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-3, 30),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-3, 30),
        "min_split_gain": trial.suggest_float("min_split_gain", 0, 15),
    }

    return 